In [1]:
import requests
import re
import logging
import io

from razdel import sentenize
from typing import Union
from bs4 import BeautifulSoup

Формируем структуру логов

In [2]:
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(name)s %(levelname)s:%(message)s')
logger = logging.getLogger(__name__)

Формируем гугл запрос

In [3]:
class Questions(object):
    def __init__(self):
        logger.info('Инициализация класса Question')

    def get_questions(self,
                      key: Union[str, list],
                      count: Union[int, list],
                      file_path: str,
                      query: Union[str, list] = None):
        '''
            Метод возвращает список вопросов для собеседования 

            Parameters:
            key: Union[str, list[str]] -- Навыки, профессии, для которых мы ищем вопросы
            count: Union[int, list[int]] -- Количество необходимых вопросов 
                                            для каждого навыка, если передан int, то 
                                            используется общее для всех количество.
            file_path: str -- путь до директории, куда сохранить результаты
            query: Union[str, list[str]] = None -- можно самому передать нужные запросы,
                                                а не использовать запрос по умолчанию.
                                                Формируйте запрос полностью с использованием
                                                key. 


        '''

        coef_dop_count = 0.1  # Коэффицент для увелечения кол-ва count

        if type(key) == str:
            key = [key]

        if type(count) == int:
            count = count + count * 0.1
            count = [count] * len(key)

        if len(key) != len(count):
            logger.error('key != count. key - {}, count - {}'.format(
                len(key), len(count)))

        if query:
            if type(query) == str:
                query = [query] * len(key)
            for index, q in enumerate(query):
                if not re.search('{}', q):
                    query[index] += ' {}'
                    logger.info('Добавили {} в запрос')
                
                query[index] = query[index].format(key[index])

            q = query
        else:
            q = ['вопросы для собеседования {}'.format(skill) for skill in key]

        if len(key) != len(q):
            logger.error('key != query. key - {}, count - {}'.format(
                len(key), len(q)))

        for index, skill in enumerate(key):

            count_quest = 0
            start = 0 # позиция с какого сайта начинается видача в гугл-запросе

            all_question = []

            while len(all_question) < count[index]:
                sites = self._get_site(q[index], start) # получаем сайты из гугл-запроса
                print(q)
                question = self._get_quest_from_sites(sites) # парсим вопросы с сайта
                count_quest += len(question)
                all_question += question
                start += 10

            self._write_quest(skill, file_path, all_question) 

    def _write_quest(self, skill, file_path, all_question):
        '''
            Метод для записи вопросов в файл
            
            Parameters:
            skill -- профессия 
            file_path -- путь до директории 
            all_question -- список вопросов
        '''
        path = f'{file_path}\\{skill}.txt'

        f = io.open(path, mode='w', encoding='utf-8')

        for quest in all_question:
            f.write(quest + '\n')

        f.close()

    def _get_site(self, q: str, start: int = 0) -> list:
        '''
            Метод возвращает списков url с гугл запроса

            Parameters:
            q -- Гугл запрос
            start -- Номер начального сайта        

        '''

        base_url = 'https://www.google.com/search?q={}&start={}'.format(
            q, start)
        
        try:
            responce = requests.get(base_url)
        except OSError:
            logger.exception(f'OSError: {base_url}')
        
        soup = BeautifulSoup(responce.text, 'lxml')

        with_quest = []
        
        # Ищем нужные url и очищаем их
        for link in soup.find_all('a'):
            if link.get('href').startswith(
                    '/url') and 'google.com' not in link.get('href'):
                link = link.get('href')[7:]
                link = link[:re.search('&sa=', link).span()[0]]
                with_quest.append(link)

        return with_quest

    def _preprocess(self, sent: str):
        '''
            Метод удаляет из строки лишние символы переноса, табуляции и пробела
            и воврашает ее

            Parameters:
            sent -- строка, которую нужно преобразовать 
        
        '''
        return re.sub('[\t\n ]+', ' ', sent).strip()

    def _get_quest_from_sites(self, sites: list):
        '''
            Парсит вопросы с переданных сайтов.
            
            Parameters:
            sites -- лист url сайтов
        '''
        
        result = []

        for url in sites:
            try:
                responce = requests.get(url)
            except OSError:
                logger.exception(f'OSError: {url}')
            soup = BeautifulSoup(responce.text, 'lxml')
            text = soup.get_text()

            quest = list(sentenize(text))
            questions = [sent.text for sent in quest]

            for index, sent in enumerate(questions):
                sent = self._preprocess(sent)
                if '?' in sent:
                    if len(sent.split()) > 2:
                        result.append(sent)
                    else:
                        result.append(questions[index - 1] + ' ' + sent)

        return result

In [4]:
if __name__ == '__main__':
    quest = Questions()
    quest.get_questions(key=['c++', 'java'], count=10, file_path=r'')

2021-05-20 17:10:06,039 __main__ INFO:Инициализация класса Question
2021-05-20 17:10:06,046 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): www.google.com:443
2021-05-20 17:10:06,486 urllib3.connectionpool DEBUG:https://www.google.com:443 "GET /search?q=%D0%B2%D0%BE%D0%BF%D1%80%D0%BE%D1%81%D1%8B%20%D0%B4%D0%BB%D1%8F%20%D1%81%D0%BE%D0%B1%D0%B5%D1%81%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F%20c++&start=0 HTTP/1.1" 200 None
2021-05-20 17:10:06,985 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): itanddigital.ru:443
2021-05-20 17:10:07,079 urllib3.connectionpool DEBUG:https://itanddigital.ru:443 "GET /interviewnet HTTP/1.1" 200 None
2021-05-20 17:10:07,114 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): itanddigital.ru:443
2021-05-20 17:10:07,171 urllib3.connectionpool DEBUG:https://itanddigital.ru:443 "GET /interviewc HTTP/1.1" 200 None


['вопросы для собеседования c++', 'вопросы для собеседования java']


2021-05-20 17:10:07,207 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): blog2k.ru:443
2021-05-20 17:10:07,766 urllib3.connectionpool DEBUG:https://blog2k.ru:443 "GET /archives/3091 HTTP/1.1" 200 None
2021-05-20 17:10:07,838 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): qna.habr.com:443
2021-05-20 17:10:09,157 urllib3.connectionpool DEBUG:https://qna.habr.com:443 "GET /q/286852 HTTP/1.1" 200 None
2021-05-20 17:10:09,197 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): techrocks.ru:443
2021-05-20 17:10:09,315 urllib3.connectionpool DEBUG:https://techrocks.ru:443 "GET /2019/03/01/popular-interview-questions-on-c-plus-plus/ HTTP/1.1" 200 None
2021-05-20 17:10:09,351 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): proglib.io:443
2021-05-20 17:10:09,737 urllib3.connectionpool DEBUG:https://proglib.io:443 "GET /p/tricky-challenges-cpp HTTP/1.1" 200 None
2021-05-20 17:10:09,869 urllib3.connectionpool DEBUG:Starting new HTTPS conn

['вопросы для собеседования c++', 'вопросы для собеседования java']


2021-05-20 17:10:14,402 urllib3.connectionpool DEBUG:https://github.com:443 "GET /enhorse/java-interview HTTP/1.1" 200 None
2021-05-20 17:10:14,647 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): github.com:443
2021-05-20 17:10:15,227 urllib3.connectionpool DEBUG:https://github.com:443 "GET /enhorse/java-interview/blob/master/core.md HTTP/1.1" 200 None
2021-05-20 17:10:15,492 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): github.com:443
2021-05-20 17:10:15,988 urllib3.connectionpool DEBUG:https://github.com:443 "GET /enhorse/java-interview/blob/master/java8.md HTTP/1.1" 200 None
2021-05-20 17:10:16,224 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): github.com:443
2021-05-20 17:10:16,704 urllib3.connectionpool DEBUG:https://github.com:443 "GET /enhorse/java-interview/blob/master/io.md HTTP/1.1" 200 None
2021-05-20 17:10:16,811 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): github.com:443
2021-05-20 17:10:17,345 urllib3.c